In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("final_output.csv")

In [3]:
data.head(5)

,FTHG,FTAG,HST,AST,B365H,B365D,B365A,FTR
0,6,0,14.0,2.0,1.10,10.00,23.00,H
1,0,2,2.0,6.0,2.80,3.25,2.60,A
2,2,1,8.0,4.0,1.36,5.00,8.50,H
3,1,0,4.0,3.0,3.40,3.30,2.20,H
4,2,0,8.0,3.0,1.80,3.50,4.75,H


In [4]:
# Import the module
from sklearn.preprocessing import LabelEncoder

# Create an instance of the class
le = LabelEncoder()

# Fit the encoder on the column
le.fit(['A', 'H', 'D'])

# Check the classes and their corresponding labels
print(le.classes_)
# Output: array(['A', 'D', 'H'], dtype='<U1')

# Transform the column into numerical labels
data['FTR'] = le.transform(data['FTR'])
print(data['FTR'])
# Output: array([2, 0, 1, 1, 2, ...])



['A' 'D' 'H']
0        2
1        0
2        2
3        2
4        2
        ..
38753    2
38754    0
38755    2
38756    1
38757    2
Name: FTR, Length: 38758, dtype: int32


In [5]:
#Splitting the data into independent and dependent variables
X = data.iloc[:,4:7].values
y = data.iloc[:,7].values
print('The independent features set: ')
print(X[:7,:])
print('The dependent variable: ')
print(y[:7])

The independent features set: 
[[ 1.1  10.   23.  ]
 [ 2.8   3.25  2.6 ]
 [ 1.36  5.    8.5 ]
 [ 3.4   3.3   2.2 ]
 [ 1.8   3.5   4.75]
 [ 2.2   3.2   3.5 ]
 [ 2.3   3.4   3.1 ]]
The dependent variable: 
[2 0 2 2 2 1 2]


In [6]:
X = X.astype('float32')
X.dtype

dtype('float32')

In [7]:
X.shape

(38758, 3)

In [8]:
import torch


y = torch.tensor(y,dtype=torch.long)

In [9]:
y.shape

torch.Size([38758])

In [10]:
# Split the data to train and test dataset.
from sklearn.model_selection import train_test_split
sample_weight = np.random.RandomState(42).rand(y.shape[0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=True,random_state=42)

In [11]:
X_train.shape

(27130, 3)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence


In [13]:
import torch
import torch.nn as nn

class CustomTransformerLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout=0.1):
        super(CustomTransformerLayer, self).__init__()
        self.self_attention = nn.MultiheadAttention(hidden_dim, num_heads, dropout=dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(hidden_dim, 4 * hidden_dim),
            nn.ReLU(),
            nn.Linear(4 * hidden_dim, hidden_dim),
        )
        self.layer_norm1 = nn.LayerNorm(hidden_dim)
        self.layer_norm2 = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.self_attention(x, x, x)
        x = x + self.dropout(attn_output)
        x = self.layer_norm1(x)
        ff_output = self.feed_forward(x)
        x = x + self.dropout(ff_output)
        x = self.layer_norm2(x)
        return x

class TransformerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer_layers = nn.ModuleList([
            CustomTransformerLayer(hidden_dim, num_heads) for _ in range(num_layers)
        ])
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        for layer in self.transformer_layers:
            x = layer(x)
        x = x.mean(dim=1)
        x = self.fc(x)
        return x


In [14]:
#set random seeds
torch.manual_seed(42)

#creating a data loader
# set up DataLoader for training set
train_loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=64)

# set up DataLoader for testing set
test_loader = DataLoader(list(zip(X_test, y_test)), shuffle=True, batch_size=10)

#define hyperparameters
input_dim = 3
hidden_dim = 64
output_dim = 3
num_layers = 9
num_heads = 4
batch_size = 64
num_epochs = 25
learning_rate = 0.001

#create models and optimizers
model = TransformerModel(input_dim,hidden_dim, output_dim, num_layers, num_heads)

optimizer =  optim.Adam(model.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()


c:\Users\USER\Wayne\Rolvaag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
#define hyperparameters
for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        #adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item() 
        _,predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        total_samples += targets.size(0)

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * total_correct / total_samples

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch 1/25, Loss: 1.0476, Accuracy: 46.73%
Epoch 2/25, Loss: 1.0278, Accuracy: 48.52%
Epoch 3/25, Loss: 1.0270, Accuracy: 48.69%
Epoch 4/25, Loss: 1.0235, Accuracy: 48.75%
Epoch 5/25, Loss: 1.0217, Accuracy: 49.04%
Epoch 6/25, Loss: 1.0209, Accuracy: 48.81%
Epoch 7/25, Loss: 1.0365, Accuracy: 48.04%
Epoch 8/25, Loss: 1.0654, Accuracy: 45.39%
Epoch 9/25, Loss: 1.0604, Accuracy: 46.27%
Epoch 10/25, Loss: 1.0612, Accuracy: 45.82%
Epoch 11/25, Loss: 1.0719, Accuracy: 44.27%
Epoch 12/25, Loss: 1.0747, Accuracy: 43.84%
Epoch 13/25, Loss: 1.0748, Accuracy: 43.78%
Epoch 14/25, Loss: 1.0753, Accuracy: 43.73%
Epoch 15/25, Loss: 1.0753, Accuracy: 43.73%
Epoch 16/25, Loss: 1.0749, Accuracy: 43.73%
Epoch 17/25, Loss: 1.0750, Accuracy: 43.73%
Epoch 18/25, Loss: 1.0751, Accuracy: 43.73%
Epoch 19/25, Loss: 1.0750, Accuracy: 43.73%
Epoch 20/25, Loss: 1.0751, Accuracy: 43.73%
Epoch 21/25, Loss: 1.0750, Accuracy: 43.73%
Epoch 22/25, Loss: 1.0750, Accuracy: 43.73%
Epoch 23/25, Loss: 1.0747, Accuracy: 43.7

In [16]:
#evaluating the model on a test set
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        #adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100* correct/ total
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 44.19


In [17]:
#["B365H","B365D","B365A"]
X_new = np.array([[2.90,3.60,2.30]])
#X_net = torch.FloatTensor(X_new)
#X_new.view(-1,1)
X_new = torch.IntTensor(X_new)
X_new.shape

torch.Size([1, 3])

In [18]:
import torch

# Assuming your model, optimizer, and criterion are already defined
# model = TransformerModel(input_dim, hidden_dim, output_dim, num_layers, num_heads)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# Load the trained model checkpoint
#checkpoint = torch.load('path_to_your_model_checkpoint.pth')
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']

# Set the model to evaluation mode
model.eval()

# Create a new input tensor with 7 elements
new_input = torch.tensor([12.96,3.23,1.90])

# Adjust the dimensionality of the input data
new_input = new_input.unsqueeze(0).unsqueeze(1)  # Adding batch and sequence dimensions

# Make the prediction
with torch.no_grad():
    prediction = model(new_input)

# Convert the prediction to a probability distribution using softmax
predicted_probs = torch.softmax(prediction, dim=1)

# Get the predicted class
predicted_class = torch.argmax(predicted_probs, dim=1).item()

print("Predicted Class:", predicted_class)
#{0:A, 1:D, 2:H}


Predicted Class: 2
